<a href="https://colab.research.google.com/github/dnmalan/advanced-data-journalism-23/blob/main/2_More_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. More with Pandas



In this lesson, we will continue working with pandas oh our dataset of baseball player salaries, which includes columns for Name, Team, League, Position, and Salary. We will cover the following concepts and functions that are very useful in data analysis:
- Rank
- ISIN
- query
- groupby (with count, sum, and average)
- merging the dataset with a different dataset (joining)



##2.1 Rank

 Ranking allows us to assign a rank or position to each data point within a specified column based on its value. We'll use the rank() function in Pandas to achieve this.

 First, we'll import pandas and the dataset.

In [ ]:
import pandas as pd

# Import data from a CSV file
df = pd.read_csv('https://raw.githubusercontent.com/dnmalan/advanced-data-journalism-23/main/data/baseball_players.csv')

# Display the first few rows of the DataFrame
df.head()


**Rank by salary**

To rank the data based on a specific column, such as Salary, you can use the rank() function. By default, it ranks in ascending order (1 being the lowest value).

We'll create a new column called "Salarly_rank" to store the ranked values.


In [ ]:
# Rank the players by Salary in ascending order
df['Salary_Rank'] = df['Salary'].rank()

# Display the first few rows with the Salary ranks
df.head()


**Rank in descending order**

If you want to rank in descending order (1 being the highest value), you can use the ascending parameter.

In [ ]:
# Rank the players by Salary in descending order
df['Salary_Rank_Desc'] = df['Salary'].rank(ascending=False)

# Display the first few rows with the descending Salary ranks
df.head()


Handling ties

When there are tied values, you can choose how to handle them using the method parameter. The default method is 'average,' which assigns the average rank to tied values. Other methods include 'min,' 'max,' and 'dense.'

In [ ]:
# Rank the players by Salary, using the 'min' method to handle ties
df['Salary_Rank_Min'] = df['Salary'].rank(method='min')

# Display the first few rows with the Salary ranks using the 'min' method
df[['Name', 'Salary', 'Salary_Rank_Min']].head()

##2.2 ISIN

The isin function allows us to filter rows based on whether a specific value is present in a column. THis allows us to write more powerful filters much more quickly than using the filter method we looked at in Lesson 1.3.

As a reminder, here's what that method looks like:

**df[df['Team'] == 'Kansas City Royals']**

If you want to filter by multiple teams, you would use this syntax:

**df[(condition1) | (condition2)]**

Which looks like this:


In [ ]:
   df[(df['Team'] == 'Kansas City Royals') | (df['Team'] == 'New York Yankees')]

Now let's use ISIN to write this in a much more efficient way.

In [ ]:
 #filter for players on two teams using ISIN

 df[df['Team'].isin(['Kansas City Royals', 'New York Yankees'])]

To save the filtered players, create a new dataframe.

In [ ]:
# create a df of the selected players and save it into a new dataframe

selected_players =  df[df['Team'].isin(['Kansas City Royals', 'New York Yankees'])]

# view new dataframe
selected_players.head()

You can also create a list of the values you want to filter for. This is especially helpful when you have many different values to search for and is a generally cleaner way to write code.

In [ ]:
# create a list of selected teams
selected_teams = ['New York Yankees', 'Boston Red Sox']

# filter for selected teams and save into a new dataframe
selected_players = df[df['Team'].isin(selected_teams)]

# view new dataframe
selected_players.head()

##2.3 Group by

The groupby function allows us to group data by one or more columns and then apply aggregation functions to the grouped data.

First, let's group by the teams. Then, we'll use count, sum, and mean as examples.


In [ ]:
# Group players by team
# this creates an object, not a dataframe, so the printout is weird but lets us know this is working

df.groupby('Team')

In [ ]:
# now add an aggregation. let's start with count
# this syntax will group by team, then count how many values are in every column

df.groupby('Team').count()

Using the agg() function allows us to aggregate by a certain column and also by multiple columns


In [ ]:
df.groupby('Team').agg({'Salary': 'count'})

In [ ]:
# Group players by team and calculate the count, sum, and average salary for each team, and store them into a new dataframe
team_salary_stats = df.groupby('Team').agg({'Salary': ['count', 'sum', 'mean']}).reset_index()

# Create better column names for the new dataframe
team_salary_stats.columns = ['Team', 'Player_Count', 'Total_Salary', 'Average_Salary']

#print entire table
print(team_salary_stats)


##2.4 Query

The query function allows us to filter rows using a more expressive syntax. We can also use it to compare values.

Let's use it to filter for numbers above a certain threshold. (This is similar to filtering we did in lesson 1.3)

First, we're going to import a new dataset of Olympics medals by country.

In [ ]:
# Import data from a CSV file
df_olympics = pd.read_csv('https://raw.githubusercontent.com/dnmalan/advanced-data-journalism-23/main/data/olympics_medals_country_wise.csv')

# Display the first few rows of the DataFrame
df_olympics.head()

Let's get to know the new dataset a little bit.

In [ ]:
# how many rows and columns
df_olympics.shape

In [ ]:
# check data types

df_olympics.dtypes

In [ ]:
# statistical overview of numerical columns

df_olympics.describe()

Now let's find countries with more than 100 summer gold medals.

In [ ]:
# use query to find countries iwth more than 100 summer_gold

high_summer_golds = df_olympics.query('summer_gold > 100')

# preview results

high_summer_golds.head()

In [ ]:
# how many countries have more than 100 summer gold medals?
# this syntax of [0] with shape will show you only the number of rows and not the number of columns

high_summer_golds.shape[0]

Now the really cool part! We can use query to filter the dataset based on a comparison. For example, we can search for countries who have more winter gold medals than summer gold medals.



In [ ]:
df_olympics.query('winter_gold > summer_gold')

##2.5 Merging datasets

The merge function in Pandas is a powerful and versatile tool for combining and joining two or more DataFrames based on common columns or indices. It is similar to the SQL JOIN operation and allows you to merge data from different sources into a single DataFrame.

Here's the basic syntax of the merge function:

**merged_df = df1.merge(df2, on='column')**

Using the olympic medals dataset, let's say we have one dataset of all the medal counts, but with only the country code. Then we have another dataset showing only the country code and corresponding country.

In [ ]:
# import two datasets and look at the first few rows

# Import country medals from a CSV file
# this will be our "main" dataset
country_medals = pd.read_csv('https://raw.githubusercontent.com/dnmalan/advanced-data-journalism-23/main/data/country_medals.csv')

country_medals.head()

In [ ]:
# import country codes from a CSV file
# this will be our "lookup" dataset
country_codes = pd.read_csv('https://raw.githubusercontent.com/dnmalan/advanced-data-journalism-23/main/data/country_codes.csv')

country_codes.head()

With our datasets imported, we can now merge them in one quick line of code. Each country code will match up, showing the complete rows from both datasets with the country name, code, and medal counts.

In [ ]:
merged_data = country_medals.merge(country_codes, on='ioc_code ')

merged_data.head()

###More on combining data

This was a very basic merge (aka, join). Joins can really be much more complicated, depending on your data and goals. There are also many problems that can happen during a join, and also more functions that combine data in similar ways to merging!

For more on joining using pandas, see this great explanation of using merge(), join(), and concatenate() on [RealPython.com](https://realpython.com/pandas-merge-join-and-concat/).